In [1]:
from pyspark.sql import SparkSession
import os

# Attach Delta Lake JAR to Spark
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages io.delta:delta-spark_2.12:3.1.0 pyspark-shell"
)

spark = (
    SparkSession.builder
    .appName("FeatureStore-Bronze-Ingestion")
    .master("local[*]")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog"
    )
    .getOrCreate()
)

spark


25/12/28 07:46:45 WARN Utils: Your hostname, RidhiGuntur resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/12/28 07:46:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ridhi/feature-store-delta/venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ridhi/.ivy2/cache
The jars for the packages stored in: /home/ridhi/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8493a1d3-be12-414a-ba69-79d472d255b7;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 562ms :: artifacts dl 26ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.1.0 from central in [default]
	io.delta#delta-storage;3.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

In [2]:
spark.range(5).show()


[Stage 0:>                                                        (0 + 12) / 12]

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [3]:
raw_df = (
    spark.read
    .option("header", True)
    .option("inferSchema", True)
    .csv("../data/raw/user_events.csv")
)

raw_df.show()
raw_df.printSchema()


+-------+----------+-----------+-------------------+
|user_id|event_type|event_value|         event_time|
+-------+----------+-----------+-------------------+
|      1|     click|          5|2024-01-01 10:00:00|
|      1|     click|          3|2024-01-02 11:00:00|
|      2|      view|          1|2024-01-01 09:30:00|
|      2|     click|          2|2024-01-03 14:00:00|
|      3|      view|          4|2024-01-02 16:45:00|
+-------+----------+-----------+-------------------+

root
 |-- user_id: integer (nullable = true)
 |-- event_type: string (nullable = true)
 |-- event_value: integer (nullable = true)
 |-- event_time: timestamp (nullable = true)



In [4]:
raw_df.write \
    .format("delta") \
    .mode("overwrite") \
    .save("../delta/bronze/user_events")


25/12/28 07:47:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

In [5]:
bronze_df = spark.read.format("delta") \
    .load("../delta/bronze/user_events")

bronze_df.show()


+-------+----------+-----------+-------------------+
|user_id|event_type|event_value|         event_time|
+-------+----------+-----------+-------------------+
|      1|     click|          5|2024-01-01 10:00:00|
|      1|     click|          3|2024-01-02 11:00:00|
|      2|      view|          1|2024-01-01 09:30:00|
|      2|     click|          2|2024-01-03 14:00:00|
|      3|      view|          4|2024-01-02 16:45:00|
+-------+----------+-----------+-------------------+



In [7]:
bronze_path = "/home/ridhi/feature-store-delta/delta/bronze/user_events"

spark.sql(
    f"DESCRIBE HISTORY delta.`{bronze_path}`"
).show(truncate=False)


+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                   |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|1      |2025-12-28 07:47:28.854|NULL  |NULL    |WRITE    |{mode -> Overwrite, partitionBy -> []}|NULL|NULL    |NULL     |0          |Serializable  |false        |{numFiles -> 1, nu